<h1>Energy: Preprocessing</h1>

In [1]:
import numpy as np
import pandas as pd

<h2>Data Exploration - Energy Dataset</h2>

In [2]:
df_raw = pd.read_csv("energy_dataset.csv")

print(df_raw.head())

print(f"\nDataframe shape: {df_raw.shape}")

                        time  generation biomass  \
0  2015-01-01 00:00:00+01:00               447.0   
1  2015-01-01 01:00:00+01:00               449.0   
2  2015-01-01 02:00:00+01:00               448.0   
3  2015-01-01 03:00:00+01:00               438.0   
4  2015-01-01 04:00:00+01:00               428.0   

   generation fossil brown coal/lignite  generation fossil coal-derived gas  \
0                                 329.0                                 0.0   
1                                 328.0                                 0.0   
2                                 323.0                                 0.0   
3                                 254.0                                 0.0   
4                                 187.0                                 0.0   

   generation fossil gas  generation fossil hard coal  generation fossil oil  \
0                 4844.0                       4821.0                  162.0   
1                 5196.0                       4755.

<h3>Checking and Removing Data Redundancy</h3>

In [3]:
# Counting the number of values in each column that are not 0 or null:
non_zero_or_null_count = [(col, df_raw[df_raw[col].notnull() & df_raw[col].ne(0)].shape[0]) for col in df_raw.columns]

df_non_zero_null_count = pd.DataFrame(non_zero_or_null_count, columns=["Column Name", "Non Zero/Null Count"])

print(df_non_zero_null_count)

#It can be seen that 8 columns contain no useful data. 

unnecessary_columns = df_non_zero_null_count[df_non_zero_null_count["Non Zero/Null Count"] == 0]["Column Name"].tolist()

print(f"\nColumns with no useful data: {unnecessary_columns}")
print(f"\nNumber of non-useful columns: {len(unnecessary_columns)}.")

                                    Column Name  Non Zero/Null Count
0                                          time                35064
1                            generation biomass                35041
2          generation fossil brown coal/lignite                24529
3            generation fossil coal-derived gas                    0
4                         generation fossil gas                35045
5                   generation fossil hard coal                35043
6                         generation fossil oil                35042
7                   generation fossil oil shale                    0
8                        generation fossil peat                    0
9                         generation geothermal                    0
10   generation hydro pumped storage aggregated                    0
11  generation hydro pumped storage consumption                22438
12   generation hydro run-of-river and poundage                35042
13             generation hydro wa

In [4]:
null_count = df_raw.isnull().sum()

print("Null values in each column: \n")
print(null_count)

#The number of null values in each column is checked here and correlates to the number of non-zero
# ... values found in the columns that contain no useful data above. 

Null values in each column: 

time                                               0
generation biomass                                19
generation fossil brown coal/lignite              18
generation fossil coal-derived gas                18
generation fossil gas                             18
generation fossil hard coal                       18
generation fossil oil                             19
generation fossil oil shale                       18
generation fossil peat                            18
generation geothermal                             18
generation hydro pumped storage aggregated     35064
generation hydro pumped storage consumption       19
generation hydro run-of-river and poundage        19
generation hydro water reservoir                  18
generation marine                                 19
generation nuclear                                17
generation other                                  18
generation other renewable                        18
generation solar

<h3>Removing unnecessary columns</h3>

In [5]:
df_raw2 = df_raw.drop(unnecessary_columns, axis=1)

print(df_raw2.head())

                        time  generation biomass  \
0  2015-01-01 00:00:00+01:00               447.0   
1  2015-01-01 01:00:00+01:00               449.0   
2  2015-01-01 02:00:00+01:00               448.0   
3  2015-01-01 03:00:00+01:00               438.0   
4  2015-01-01 04:00:00+01:00               428.0   

   generation fossil brown coal/lignite  generation fossil gas  \
0                                 329.0                 4844.0   
1                                 328.0                 5196.0   
2                                 323.0                 4857.0   
3                                 254.0                 4314.0   
4                                 187.0                 4130.0   

   generation fossil hard coal  generation fossil oil  \
0                       4821.0                  162.0   
1                       4755.0                  158.0   
2                       4581.0                  157.0   
3                       4131.0                  160.0   
4    

<h3>Interpolating null values</h3>

In [6]:
# Linear interpolation is used as an approximation. Since the number of null values is no more than ~0.1%
# in any column, this seems appropriate. 
df_interpolated = df_raw2.interpolate(method="linear", limit_direction="both")
df_rounded = df_interpolated.round(1)

null_count_interpolated = df_rounded.isnull().sum()

print("Null values in each column: \n")
print(null_count_interpolated)

Null values in each column: 

time                                           0
generation biomass                             0
generation fossil brown coal/lignite           0
generation fossil gas                          0
generation fossil hard coal                    0
generation fossil oil                          0
generation hydro pumped storage consumption    0
generation hydro run-of-river and poundage     0
generation hydro water reservoir               0
generation nuclear                             0
generation other                               0
generation other renewable                     0
generation solar                               0
generation waste                               0
generation wind onshore                        0
forecast solar day ahead                       0
forecast wind onshore day ahead                0
total load forecast                            0
total load actual                              0
price day ahead                        

<h3>Checking Duplicate Rows</h3>

In [7]:
duplicate_rows = df_rounded.duplicated()
print(f"Number of duplicate rows =  {duplicate_rows.sum()}")

Number of duplicate rows =  0


<h3>Exporting Preprocessed Dataset</h3>

In [8]:
# mv == multivariable
energy_df_mv = df_rounded[["time", "total load actual", "generation solar", "generation wind onshore" ]]

energy_df_mv.to_csv("energy_mv.csv", index=False)